# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [1]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [7]:
# 1) Seu código aqui
url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Munic%C3%ADpio'

In [9]:
# 2) Seu código aqui
response = requests.get(url)
data = response.json()

# Passo 2: Carregar os dados em um DataFrame do pandas
df = pd.json_normalize(data['items'])

# Passo 3: Analisar o DataFrame para encontrar os três status das solicitações mais frequentes
status_counts = df['status'].value_counts().nlargest(3)

# Mostrar os três status mais frequentes e suas frequências
print(status_counts)

status
Deferido        106
Arquivado        53
Regularizado     26
Name: count, dtype: int64


In [11]:
# 3) Seu código aqui

# Transformar a coluna data_status em string
df['data_status'] = df['data_status'].astype(str)

# Extrair o ano usando split
df['ano_status'] = df['data_status'].str.split('-').str[0]

# Verificar as primeiras linhas do DataFrame para garantir que a nova coluna foi criada corretamente
print("\nExemplo de dados com a nova coluna 'ano_status':")
print(df[['data_status', 'ano_status']].head())


Exemplo de dados com a nova coluna 'ano_status':
  data_status  ano_status
0  27/06/2024  27/06/2024
1  05/10/2023  05/10/2023
2  27/09/2019  27/09/2019
3  21/10/2022  21/10/2022
4  18/05/2012  18/05/2012


In [13]:
# 4) Seu código aqui
# Contar a frequência de cada ano na coluna ano_status
ano_counts = df['ano_status'].value_counts()
print("\nFrequência de cada ano no campo 'ano_status':")
print(ano_counts)


Frequência de cada ano no campo 'ano_status':
ano_status
25/05/2007    6
02/08/2007    5
30/10/2008    5
27/09/2019    4
15/09/2008    4
             ..
13/02/2015    1
28/08/2023    1
20/02/2014    1
08/09/2022    1
12/03/2009    1
Name: count, Length: 203, dtype: int64


####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [15]:
#1) Seu código aqui
def consulta_dados(uf, tipo_interessado):
    """
    Consulta dados do Tesouro Nacional com base no UF e tipo de interessado.

    Parâmetros:
    uf (str): Unidade Federativa (por exemplo, 'RJ', 'SP').
    tipo_interessado (str): Tipo de interessado ('Estado' ou 'Município').

    Retorna:
    DataFrame: Dados da consulta no formato DataFrame.
    """
    # Construir a URL dinamicamente
    base_url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl'
    url = f"{base_url}?uf={uf}&tipo_interessado={tipo_interessado}"
    
    # Fazer a solicitação HTTP para a URL construída
    response = requests.get(url)
    data = response.json()
    
    # Carregar os dados em um DataFrame do pandas
    df = pd.json_normalize(data['items'])
    
    return df

# Exemplo de uso
df_sp_estado = consulta_dados('SP', 'Estado')
print("Consulta para SP, Estado:")
print(df_sp_estado.head())

df_rj_municipio = consulta_dados('RJ', 'Município')
print("\nConsulta para RJ, Município:")
print(df_rj_municipio.head())

Consulta para SP, Estado:
   id_pleito tipo_interessado interessado  cod_ibge  uf               num_pvl  \
0      48051           Estado   São Paulo        35  SP  PVL02.002802/2021-77   
1      11493           Estado   São Paulo        35  SP                  None   
2      11263           Estado   São Paulo        35  SP                  None   
3      11982           Estado   São Paulo        35  SP                  None   
4      11169           Estado   São Paulo        35  SP                  None   

                                              status          num_processo  \
0  Encaminhado à PGFN com manifestação técnica fa...  17944.103753/2021-51   
1                                           Deferido  17944.001461/2008-16   
2                                          Arquivado  17944.001396/2011-16   
3                                           Deferido  17944.001573/2009-40   
4  Encaminhado à PGFN com manifestação técnica fa...  17944.001375/2012-81   

         data_prot

In [47]:
# 2) Seu código aqui
# Consultar os dados para Minas Gerais (MG) e tipo de interessado 'Estado'
df_mg_estado = consulta_dados('MG', 'Estado')

# Verificar as primeiras linhas do DataFrame para garantir que os dados estão corretos
print("Primeiras linhas do DataFrame para MG e tipo 'Estado':")
print(df_mg_estado.head())

# Verificar os valores únicos na coluna 'status'
print("\nValores únicos na coluna 'status':")
print(df_mg_estado['status'].unique())

# Filtrar os dados pelo status 'Arquivado por decurso de prazo'
status_desejado = 'Arquivado por decurso de prazo'
df_filtrado = df_mg_estado[df_mg_estado['status'].str.contains(status_desejado, na=False)]

# Contar o número de solicitações com o status desejado
quantidade = df_filtrado.shape[0]

# Mostrar a quantidade de solicitações
print(f"\nQuantidade de solicitações para MG com status '{status_desejado}': {quantidade}")

Primeiras linhas do DataFrame para MG e tipo 'Estado':
   id_pleito tipo_interessado   interessado  cod_ibge  uf num_pvl  \
0       7032           Estado  Minas Gerais        31  MG    None   
1      11054           Estado  Minas Gerais        31  MG    None   
2      10338           Estado  Minas Gerais        31  MG    None   
3       7435           Estado  Minas Gerais        31  MG    None   
4       9197           Estado  Minas Gerais        31  MG    None   

                                              status          num_processo  \
0  Encaminhado à PGFN com manifestação técnica fa...  17944.000482/2012-92   
1  Encaminhado à PGFN com manifestação técnica fa...  17944.001346/2012-10   
2  Encaminhado à PGFN com manifestação técnica fa...  17944.001175/2009-23   
3  Encaminhado à PGFN com manifestação técnica fa...  17944.000576/2009-66   
4  Encaminhado à PGFN com manifestação técnica fa...  17944.000924/2009-03   

         data_protocolo                                      

In [49]:
# 3) Seu código aqui
# Consultar os dados para Bahia (BA) e tipo de interessado 'Município'
df_ba_municipio = consulta_dados('BA', 'Município')

# Tentar filtrar os dados pelo status 'Deferido'
status_desejado = 'Deferido'
df_deferido = df_ba_municipio[df_ba_municipio['status'].str.contains(status_desejado, na=False)]

# Verificar as colunas novamente após o filtro
print("\nColunas após o filtro:")
print(df_deferido.columns)

# Contar as solicitações deferidas por município, utilizando a coluna 'interessado'
municipio_counts = df_deferido['interessado'].value_counts()

# Identificar o município com o maior número de solicitações deferidas
municipio_mais_deferido = municipio_counts.idxmax()
quantidade_deferidos = municipio_counts.max()

# Mostrar o município com o maior número de solicitações deferidas e a quantidade
print(f"\nMunicípio da Bahia com mais solicitações deferidas: {municipio_mais_deferido} com {quantidade_deferidos} solicitações deferidas")


Colunas após o filtro:
Index(['id_pleito', 'tipo_interessado', 'interessado', 'cod_ibge', 'uf',
       'num_pvl', 'status', 'num_processo', 'data_protocolo', 'tipo_operacao',
       'finalidade', 'tipo_credor', 'credor', 'moeda', 'valor',
       'pvl_assoc_divida', 'pvl_contradado_credor', 'data_status'],
      dtype='object')

Município da Bahia com mais solicitações deferidas: Luís Eduardo Magalhães com 16 solicitações deferidas


In [45]:
# 4) Seu código aqui
# Consultar os dados para Bahia (BA) e tipo de interessado 'Estado'
df_ba_estado = consulta_dados('BA', 'Estado')

# Verificar as colunas do DataFrame
print("Colunas do DataFrame:")
print(df_ba_estado.columns)

# Mostrar as primeiras linhas do DataFrame para inspeção
print("\nPrimeiras linhas do DataFrame:")
print(df_ba_estado.head())

# Salvar o DataFrame em um arquivo .csv
df_ba_estado.to_csv('solicitacoes_bahia_estado.csv', index=False)

print("\nArquivo 'solicitacoes_bahia_estado.csv' salvo com sucesso.")

Colunas do DataFrame:
Index(['id_pleito', 'tipo_interessado', 'interessado', 'cod_ibge', 'uf',
       'num_pvl', 'status', 'num_processo', 'data_protocolo', 'tipo_operacao',
       'finalidade', 'tipo_credor', 'credor', 'moeda', 'valor',
       'pvl_assoc_divida', 'pvl_contradado_credor', 'data_status'],
      dtype='object')

Primeiras linhas do DataFrame:
   id_pleito tipo_interessado interessado  cod_ibge  uf num_pvl  \
0      18550           Estado       Bahia        29  BA    None   
1      10431           Estado       Bahia        29  BA    None   
2      18441           Estado       Bahia        29  BA    None   
3      18583           Estado       Bahia        29  BA    None   
4       9412           Estado       Bahia        29  BA    None   

                                              status          num_processo  \
0                                           Deferido  19407.000048/2006-77   
1  Encaminhado à PGFN com manifestação técnica fa...  17944.001198/2012-33   
2  